In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import pandas as pd

from sklearn.metrics import precision_score, recall_score

In [16]:
df = pd.read_csv('../3_PreprocessingMaleDS/final.csv')

In [17]:
labels_usage = df['usage']
labels_articleType = df['articleType']

In [18]:
image_paths = [f'../3_PreprocessingMaleDS/finalDS/{str(img_id)}.jpg'  for img_id in df['id']]
images = [np.array(Image.open(img_path).resize((80, 60))) for img_path in image_paths]
# images = [np.array(Image.open(img_path).resize((224, 224))) for img_path in image_paths] # for transfer learning

In [19]:
images = np.array(images)
labels_usage = np.array(labels_usage)
labels_articleType = np.array(labels_articleType)

In [20]:
label_encoder = LabelEncoder()
labels_usage = label_encoder.fit_transform(labels_usage)
labels_articleType = label_encoder.fit_transform(labels_articleType)

# CNN From Scratch For "usage" 

In [8]:
X_train, X_val, y_train, y_val = train_test_split(images, labels_usage, test_size=0.2, random_state=42)

In [10]:
model_usage = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 80, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: 'Informal' and 'Formal'
])

In [11]:
model_usage.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model_usage.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
272/272 [==============================] - 44s 157ms/step - loss: 1.0532 - accuracy: 0.8897 - val_loss: 0.2188 - val_accuracy: 0.9179
Epoch 2/20
272/272 [==============================] - 45s 165ms/step - loss: 0.2000 - accuracy: 0.9240 - val_loss: 0.1610 - val_accuracy: 0.9396
Epoch 3/20
272/272 [==============================] - 50s 183ms/step - loss: 0.1757 - accuracy: 0.9322 - val_loss: 0.1807 - val_accuracy: 0.9368
Epoch 4/20
272/272 [==============================] - 50s 183ms/step - loss: 0.1689 - accuracy: 0.9389 - val_loss: 0.1528 - val_accuracy: 0.9419
Epoch 5/20
272/272 [==============================] - 49s 182ms/step - loss: 0.1520 - accuracy: 0.9453 - val_loss: 0.1621 - val_accuracy: 0.9410
Epoch 6/20
272/272 [==============================] - 49s 182ms/step - loss: 0.1501 - accuracy: 0.9455 - val_loss: 0.1585 - val_accuracy: 0.9414
Epoch 7/20
272/272 [==============================] - 49s 180ms/step - loss: 0.1357 - accuracy: 0.9496 - val_loss: 0.1516 - val_ac

In [13]:
predicted_probabilities = model_usage.predict(X_val)
predicted_labels = np.argmax(predicted_probabilities, axis=1)

precision = precision_score(y_val, predicted_labels)
recall = recall_score(y_val, predicted_labels)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


68/68 [==============================] - 3s 45ms/step
Precision: 0.8419
Recall: 0.8383


In [14]:
model_usage.save('cnn_usage.h5')

C:\Users\Electro Store\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# CNN From Scratch For "articleType"

In [21]:
X_train, X_val, y_train, y_val = train_test_split(images, labels_articleType, test_size=0.2, random_state=42)

In [22]:
model_articleType = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 80, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes: 'shirt', 'tshirt' and 'trouser'
])

In [23]:
model_articleType.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model_articleType.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
272/272 [==============================] - 43s 155ms/step - loss: 1.3556 - accuracy: 0.7225 - val_loss: 0.3635 - val_accuracy: 0.8456
Epoch 2/20
272/272 [==============================] - 45s 164ms/step - loss: 0.4322 - accuracy: 0.8137 - val_loss: 0.2694 - val_accuracy: 0.8981
Epoch 3/20
272/272 [==============================] - 46s 168ms/step - loss: 0.3343 - accuracy: 0.8555 - val_loss: 0.2516 - val_accuracy: 0.9059
Epoch 4/20
272/272 [==============================] - 46s 168ms/step - loss: 0.2807 - accuracy: 0.8860 - val_loss: 0.2001 - val_accuracy: 0.9272
Epoch 5/20
272/272 [==============================] - 47s 173ms/step - loss: 0.2551 - accuracy: 0.8978 - val_loss: 0.2019 - val_accuracy: 0.9313
Epoch 6/20
272/272 [==============================] - 47s 173ms/step - loss: 0.2216 - accuracy: 0.9140 - val_loss: 0.2010 - val_accuracy: 0.9262
Epoch 7/20
272/272 [==============================] - 46s 169ms/step - loss: 0.2154 - accuracy: 0.9147 - val_loss: 0.1506 - val_ac

In [32]:
# Get model predictions on the validation dataset
predicted_probabilities = model_articleType.predict(X_val)
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Calculate precision and recall
precision = precision_score(y_val, predicted_labels, average='weighted')
recall = recall_score(y_val, predicted_labels, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


68/68 [==============================] - 3s 41ms/step
Precision: 0.9672
Recall: 0.9668


In [26]:
model_articleType.save('cnn_articleType.h5')

C:\Users\Electro Store\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
